In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from utils.draw import draw_squares
from utils.square import SquareDataset
from torch.utils.data import DataLoader

In [56]:
class LinearModel(nn.Module):
    def __init__(self, x, y):
        super(LinearModel, self).__init__()
        self.layer1 = nn.Linear(x, y)

    def forward(self, x):
        x = self.layer1(x)
        return F.softmax(x, dim=1)

In [57]:
squares = SquareDataset(3000)
for i in range(5):
    print(squares[i])

(tensor([147.,  63.,  50., 243., 253., 194., 175.,  53.,  26.]), tensor([0., 1., 0.]))
(tensor([ 32., 208.,  76.,  96.,   8., 206., 229., 160., 180.]), tensor([0., 0., 1.]))
(tensor([ 56., 145., 213., 226., 229.,  14., 115., 103., 111.]), tensor([0., 1., 0.]))
(tensor([ 64., 219., 240.,  69., 108., 237., 224., 205., 232.]), tensor([0., 0., 1.]))
(tensor([ 27., 196., 199., 180., 190.,  33., 231., 185., 222.]), tensor([0., 0., 1.]))


# Setting things up!

In [58]:
device = torch.device('cuda')

# Use the nn package to define our model and loss function.
model = LinearModel(9, 3)
model = model.to(device)

cost = torch.nn.BCELoss()

# optimizer which Tensors it should update.
learning_rate = 1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# dataset!
dataloader = DataLoader(squares, batch_size=128)

epochs = 350

# The Optimization Loop

In [83]:
for t in range(epochs):
    for batch, (X, Y) in enumerate(dataloader):
        X, Y = X.to(device) / 255, Y.to(device)
        optimizer.zero_grad()
        pred = model(X)
        loss = cost(pred, Y)
        loss.backward()
        optimizer.step()

    if t % 50 == 0:
        print('l: {:>8f}, (e {:>3})'.format(loss.item(), t))
        
print('l: {:>8f}, (e {:>3})\nDone!'.format(loss.item(), t))

l: 0.081167, (e   0)
l: 0.078037, (e  50)
l: 0.075253, (e 100)
l: 0.072756, (e 150)
l: 0.070501, (e 200)
l: 0.068451, (e 250)
l: 0.066578, (e 300)
l: 0.064891, (e 349)
Done!


In [84]:
print("W's and b's:")
for p in model.parameters():
    print(p)

W's and b's:
Parameter containing:
tensor([[ 6.9096,  6.9024,  6.7417, -4.4641, -4.0440, -4.3879, -4.7592, -4.7063,
         -4.4715],
        [-4.4005, -4.5857, -4.2385,  7.0906,  7.2895,  7.5260, -4.9837, -5.0253,
         -4.6258],
        [-4.7008, -4.6695, -4.2674, -4.7429, -4.7396, -4.3155,  7.1942,  6.8867,
          7.2619]], device='cuda:0', requires_grad=True)
Parameter containing:
tensor([0.3625, 0.0536, 0.1359], device='cuda:0', requires_grad=True)


# Trying it out (inference)

In [99]:
img = [255, 255, 145, 34, 21, 0, 232, 32, 255]
with torch.no_grad():
    o = model(torch.tensor(img, dtype=torch.float).view(1, 9).to(device) / 255)
    
print(o)
print('\ntop:    {:>8f}\nmiddle: {:>8f}\nbottom: {:>8f}'.format(*o[0]))

tensor([[9.9666e-01, 1.4225e-12, 3.3351e-03]], device='cuda:0')

top:    0.996665
middle: 0.000000
bottom: 0.003335
